### Get the Personalize boto3 Client-1

In [0]:
import boto3

import json
import numpy as np
import pandas as pd
import time

!wget -N https://s3-us-west-2.amazonaws.com/personalize-cli-json-models/personalize.json
!wget -N https://s3-us-west-2.amazonaws.com/personalize-cli-json-models/personalize-runtime.json
!aws configure add-model --service-model file://`pwd`/personalize.json --service-name personalize
!aws configure add-model --service-model file://`pwd`/personalize-runtime.json --service-name personalize-runtime

personalize = boto3.client(service_name='personalize', endpoint_url='https://personalize.us-east-1.amazonaws.com')
personalize_runtime = boto3.client(service_name='personalize-runtime', endpoint_url='https://personalize-runtime.us-east-1.amazonaws.com')

### Specify a Bucket and Data Output Location

In [0]:
bucket = "personalize-demo"           # replace with the name of your S3 bucket
filename = "DEMO-movie-lens-100k.csv"

### Download, Prepare, and Upload Training Data

#### Download and Explore the Dataset

In [0]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data

Archive:  ml-100k.zip
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


,USER_ID,ITEM_ID,RATING,TIMESTAMP
0,196,242,3,881250949
1,186,302,3,891717742
...,...,...,...,...
99998,13,225,2,882399156
99999,12,203,3,879959583


#### Prepare and Upload Data

In [0]:
data = data[data['RATING'] > 3.6]                # keep only movies rated 3.6 and above
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data.to_csv(filename, index=False)

boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

### Create Schema

In [0]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "DEMO-schema",
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print json.dumps(create_schema_response, indent=2)

{
  "schemaArn": "arn:aws:personalize:us-east-1:237539672711:schema/DEMO-schema", 
  "ResponseMetadata": {
    "RetryAttempts": 0, 
    "HTTPStatusCode": 200, 
    "RequestId": "12eb7cba-2b64-4be9-9f6e-eeebff7629a5", 
    "HTTPHeaders": {
      "date": "Tue, 04 Dec 2018 05:49:04 GMT", 
      "x-amzn-requestid": "12eb7cba-2b64-4be9-9f6e-eeebff7629a5", 
      "content-length": "79", 
      "content-type": "application/x-amz-json-1.1", 
      "connection": "keep-alive"
    }
  }
}


### Create and Wait for Dataset Group

#### Create Dataset Group

In [0]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "DEMO-dataset-group"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print json.dumps(create_dataset_group_response, indent=2)

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:237539672711:dataset-group/DEMO-dataset-group", 
  "ResponseMetadata": {
    "RetryAttempts": 0, 
    "HTTPStatusCode": 200, 
    "RequestId": "d6b600c8-6168-4fcf-b7bf-97bd16a79aba", 
    "HTTPHeaders": {
      "date": "Tue, 04 Dec 2018 05:49:08 GMT", 
      "x-amzn-requestid": "d6b600c8-6168-4fcf-b7bf-97bd16a79aba", 
      "content-length": "99", 
      "content-type": "application/x-amz-json-1.1", 
      "connection": "keep-alive"
    }
  }
}


#### Wait for Dataset Group to Have ACTIVE Status

In [0]:
status = None
while status != "ACTIVE":
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print "DatasetGroup: {}".format(status)
    
    if status != "ACTIVE" and status != "CREATE FAILED":
        time.sleep(60)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


### Create Dataset

In [0]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

dataset_arn = create_dataset_response['datasetArn']
print json.dumps(create_dataset_response, indent=2)

{
  "ResponseMetadata": {
    "RetryAttempts": 0, 
    "HTTPStatusCode": 200, 
    "RequestId": "29ab75c8-df6e-4807-943f-1b48014181d1", 
    "HTTPHeaders": {
      "date": "Tue, 04 Dec 2018 05:50:19 GMT", 
      "x-amzn-requestid": "29ab75c8-df6e-4807-943f-1b48014181d1", 
      "content-length": "101", 
      "content-type": "application/x-amz-json-1.1", 
      "connection": "keep-alive"
    }
  }, 
  "datasetArn": "arn:aws:personalize:us-east-1:237539672711:dataset/DEMO-dataset-group/INTERACTIONS"
}


### Prepare, Create, and Wait for Dataset Import Job

#### Attach policy to S3 bucket

In [0]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy));

#### Create S3 Read Only Access Role

In [0]:
iam = boto3.client("iam")

role_name = "PersonalizeS3Role"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
);

iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
);

role_arn = create_role_response["Role"]["Arn"]
print role_arn

arn:aws:iam::237539672711:role/PersonalizeS3Role


#### Create Dataset Import Job

In [0]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "DEMO-dataset-import-job",
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print json.dumps(create_dataset_import_job_response, indent=2)

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:237539672711:dataset-import-job/DEMO-dataset-import-job", 
  "ResponseMetadata": {
    "RetryAttempts": 0, 
    "HTTPStatusCode": 200, 
    "RequestId": "3c77fe8d-d9fe-4ca5-ad03-b18e937acbb3", 
    "HTTPHeaders": {
      "date": "Tue, 04 Dec 2018 05:50:55 GMT", 
      "x-amzn-requestid": "3c77fe8d-d9fe-4ca5-ad03-b18e937acbb3", 
      "content-length": "113", 
      "content-type": "application/x-amz-json-1.1", 
      "connection": "keep-alive"
    }
  }
}


#### Wait for Dataset Import Job and Dataset Import Job Run to Have ACTIVE Status

In [0]:
status = None
while status != "ACTIVE":
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print "DatasetImportJob: {}".format(status)
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print "LatestDatasetImportJobRun: {}".format(status)
    
    if status != "ACTIVE" and status != "CREATE FAILED":
        time.sleep(60)

DatasetImportJob: CREATE PENDING
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: CREATE IN_PROGRESS
LatestDatasetImportJobRun: ACTIVE


### Select Recipe

In [0]:
recipe_list = [
    "arn:aws:personalize:::recipe/awspersonalizehrnnmodel",
    "arn:aws:personalize:::recipe/awspersonalizedeepfmmodel",
    "arn:aws:personalize:::recipe/awspersonalizesimsmodel",
    "arn:aws:personalize:::recipe/awspersonalizeffnnmodel",
    "arn:aws:personalize:::recipe/popularity-baseline"
]

recipe_arn = recipe_list[0]
print recipe_arn

arn:aws:personalize:::recipe/awspersonalizehrnnmodel


### Create and Wait for Solution

#### Create Solution

In [0]:
create_solution_response = personalize.create_solution(
    name = "DEMO-solution",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn,
    minTPS = 1
)

solution_arn = create_solution_response['solutionArn']
print json.dumps(create_solution_response, indent=2)

{
  "solutionArn": "arn:aws:personalize:us-east-1:237539672711:solution/DEMO-solution", 
  "ResponseMetadata": {
    "RetryAttempts": 0, 
    "HTTPStatusCode": 200, 
    "RequestId": "2042832f-0775-43e2-86de-53a061be1f63", 
    "HTTPHeaders": {
      "date": "Mon, 03 Dec 2018 23:55:17 GMT", 
      "x-amzn-requestid": "2042832f-0775-43e2-86de-53a061be1f63", 
      "content-length": "83", 
      "content-type": "application/x-amz-json-1.1", 
      "connection": "keep-alive"
    }
  }
}


#### Wait for Solution to Have ACTIVE Status

In [0]:
status = None
while status != "ACTIVE":
    describe_solution_response = personalize.describe_solution(
        solutionArn = solution_arn
    )
    status = describe_solution_response["solution"]["status"]
    print "Solution: {}".format(status)
    
    if status != "ACTIVE" and status != "CREATE FAILED":
        time.sleep(60)

Solution: CREATE PENDING
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE IN_PROGRESS
Solution: CREATE I

#### Get Metrics of Solution

In [0]:
get_metrics_response = personalize.get_metrics(
    solutionArn = solution_arn
)

print json.dumps(get_metrics_response, indent=2)

{
  "metrics": {
    "arn:aws:personalize:us-east-1:237539672711:model/awspersonalizehrnnmodel-c2f6287c": {
      "_user_history_length_10_pct_quantile": 11.0, 
      "_user_history_length_mean": 54.34065934065934, 
      "normalized_discounted_cumulative_gain_at_10": 0.027472527472527472, 
      "_num_unique_items": 1077.0, 
      "normalized_discounted_cumulative_gain_at_5": 0.016483516483516484, 
      "_user_history_length_50_pct_quantile": 40.0, 
      "precision_at_10": 0.005494505494505495, 
      "mean_reciprocal_rank": 0.019587187573612913, 
      "coverage": 0.4159702878365831, 
      "precision_at_25": 0.006593406593406594, 
      "precision_at_5": 0.004395604395604396, 
      "normalized_discounted_cumulative_gain_at_25": 0.055378468279180526, 
      "_user_history_length_90_pct_quantile": 120.0, 
      "_num_evaluation_users": 91.0
    }
  }, 
  "ResponseMetadata": {
    "RetryAttempts": 0, 
    "HTTPStatusCode": 200, 
    "RequestId": "5b5f4f4f-5249-4c0e-9f83-45e3fe22f09f

### Create and Wait for Campaign

#### Create campaign

In [0]:
create_campaign_response = personalize.create_campaign(
    name = "DEMO-campaign",
    solutionArn = solution_arn,
    updateMode = "MANUAL"
)

campaign_arn = create_campaign_response['campaignArn']
print json.dumps(create_campaign_response, indent=2)

{
  "campaignArn": "arn:aws:personalize:us-east-1:237539672711:campaign/DEMO-campaign", 
  "ResponseMetadata": {
    "RetryAttempts": 0, 
    "HTTPStatusCode": 200, 
    "RequestId": "527e97ba-683c-4dc7-8218-00716f22c904", 
    "HTTPHeaders": {
      "date": "Tue, 04 Dec 2018 00:54:17 GMT", 
      "x-amzn-requestid": "527e97ba-683c-4dc7-8218-00716f22c904", 
      "content-length": "83", 
      "content-type": "application/x-amz-json-1.1", 
      "connection": "keep-alive"
    }
  }
}


#### Wait for Campaign to Have ACTIVE Status

In [0]:
status = None
while status != "ACTIVE":
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print "Campaign: {}".format(status)
    
    if status != "ACTIVE" and status != "CREATE FAILED":
        time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


### Get Recommendations

#### Select a User and an Item

In [0]:
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], header=None)
items.columns = ['ITEM_ID', 'TITLE']

user_id, item_id, _ = data.sample().values[0]
item_title = items.loc[items['ITEM_ID'] == item_id].values[0][-1]
print "USER: {}".format(user_id)
print "ITEM: {}".format(item_title)

items

USER: 807
ITEM: To Kill a Mockingbird (1962)


,ITEM_ID,TITLE
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
...,...,...
1680,1681,You So Crazy (1994)
1681,1682,Scream of Stone (Schrei aus Stein) (1991)


#### Call GetRecommendations

In [0]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
    itemId = str(item_id)
)

item_list = get_recommendations_response['itemList']
title_list = [items.loc[items['ITEM_ID'] == np.int(item['itemId'])].values[0][-1] for item in item_list]

print "Recommendations: {}json.dumps(title_list, indent=2)

[
  "Sleepless in Seattle (1993)", 
  "Broken Arrow (1996)", 
  "Mrs. Doubtfire (1993)", 
  "Speed (1994)", 
  "Eraser (1996)", 
  "Schindler's List (1993)", 
  "Sound of Music, The (1965)", 
  "Field of Dreams (1989)", 
  "Mask, The (1994)", 
  "Godfather, The (1972)", 
  "Dave (1993)", 
  "Phenomenon (1996)", 
  "English Patient, The (1996)", 
  "Monty Python and the Holy Grail (1974)", 
  "Birdcage, The (1996)", 
  "Indiana Jones and the Last Crusade (1989)", 
  "Grease (1978)", 
  "E.T. the Extra-Terrestrial (1982)", 
  "Mission: Impossible (1996)", 
  "101 Dalmatians (1996)", 
  "Get Shorty (1995)", 
  "Fish Called Wanda, A (1988)", 
  "Sword in the Stone, The (1963)", 
  "True Lies (1994)", 
  "Aladdin (1992)"
]
